# Import libraries

In [1]:
from glob import glob 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
# import pickle
import random
import numpy as np
import cv2
# import datetime
from tqdm import tqdm

Missing colon in file PosixPath('/home/leeu/miniconda3/envs/torch/lib/python3.7/site-packages/matplotlib/mpl-data/matplotlibrc'), line 264 (' added by lee')
Duplicate key in file PosixPath('/home/leeu/miniconda3/envs/torch/lib/python3.7/site-packages/matplotlib/mpl-data/matplotlibrc'), line 265 ('font.family:            sans-serif')
Duplicate key in file PosixPath('/home/leeu/miniconda3/envs/torch/lib/python3.7/site-packages/matplotlib/mpl-data/matplotlibrc'), line 270 ('font.sans-serif: DejaVu Sans, Bitstream Vera Sans, Computer Modern Sans Serif, Lucida Grande, Verdana, Geneva, Lucid, Arial, Helvetica, Avant Garde, sans-serif')
Duplicate key in file PosixPath('/home/leeu/miniconda3/envs/torch/lib/python3.7/site-packages/matplotlib/mpl-data/matplotlibrc'), line 413 ('axes.unicode_minus: True  # use Unicode for the minus symbol rather than hyphen.  See')


# Retrieve images from external sources

In [7]:
# Clone the GitHub repo of mahjong tile images
!git clone https://github.com/camerash/mahjong-dataset
# mirror:
# !git clone https://github.com.cnpmjs.org/camerash/mahjong-dataset

Cloning into 'mahjong-dataset'...
fatal: unable to access 'https://github.com.cnpmjs.org/camerash/mahjong-dataset/': server certificate verification failed. CAfile: none CRLfile: none


In [8]:
# Download Describable Textures Dataset (DTD)
!wget https://www.robots.ox.ac.uk/~vgg/data/dtd/download/dtd-r1.0.1.tar.gz

--2022-01-20 13:55:31--  https://www.robots.ox.ac.uk/~vgg/data/dtd/download/dtd-r1.0.1.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 625239812 (596M) [application/x-gzip]
Saving to: 'dtd-r1.0.1.tar.gz'

dtd-r1.0.1.tar.gz     0%[                    ] 367.82K  13.5KB/s    eta 12h 10m^C


In [9]:
# Extract the DTD
!tar xf dtd-r1.0.1.tar.gz

## Load Images

In [10]:
# load background images

bg_images_filename = glob('dtd/images/*/*.jpg')
bg_images = [cv2.imread(filename) for filename in tqdm(bg_images_filename)]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5640/5640 [00:32<00:00, 174.22it/s]


In [11]:
# load mahjong image

import pandas as pd

table = pd.read_csv('mahjong-dataset/tiles-data/data.csv', index_col=False)
tile_images_filename = list(table.iloc[:, 0])
tile_images_type = list(table.iloc[:, 2])
tile_images_label = list(table.iloc[:, 1])
tile_images = [cv2.imread('mahjong-dataset/tiles-resized/' + filename) for filename in tqdm(tile_images_filename)]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 629/629 [00:01<00:00, 544.16it/s]


In [12]:
set(tile_images_type)

{'bamboo-1',
 'bamboo-2',
 'bamboo-3',
 'bamboo-4',
 'bamboo-5',
 'bamboo-6',
 'bamboo-7',
 'bamboo-8',
 'bamboo-9',
 'bonus-autumn',
 'bonus-bamboo',
 'bonus-chrysanthemum',
 'bonus-orchid',
 'bonus-plum',
 'bonus-spring',
 'bonus-summer',
 'bonus-winter',
 'characters-1',
 'characters-2',
 'characters-3',
 'characters-4',
 'characters-5',
 'characters-6',
 'characters-7',
 'characters-8',
 'characters-9',
 'dots-1',
 'dots-2',
 'dots-3',
 'dots-4',
 'dots-5',
 'dots-6',
 'dots-7',
 'dots-8',
 'dots-9',
 'honors-east',
 'honors-green',
 'honors-north',
 'honors-red',
 'honors-south',
 'honors-west',
 'honors-white'}

## Define functions

In [13]:
def image_rotate(image, angle):
    # use degree measure (e.g., 90 deg)
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)

    border_color=(255, 255, 255)

    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e. the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
 
    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
 
    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY
 
    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (nW, nH))

In [14]:
def image_resize(image, height, width, inter=cv2.INTER_AREA):
    width, height = int(width), int(height)
    
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized

In [15]:
# put fg_image on bg_image
def image_overlap(bg_image, fg_image, y_offset, x_offset):
    x_offset, y_offset = int(x_offset), int(y_offset)
    tmp = cv2.cvtColor(fg_image, cv2.COLOR_BGR2GRAY)
    
    _, alpha = cv2.threshold(tmp, 0, 255, cv2.THRESH_BINARY)
    b, g, r, a = cv2.split(cv2.cvtColor(fg_image, cv2.COLOR_RGB2RGBA))
    rgba = [b, g, r, alpha]
    fg_image = cv2.merge(rgba, 4)
    y1, y2 = y_offset, y_offset + fg_image.shape[0]
    x1, x2 = x_offset, x_offset + fg_image.shape[1]
    
    alpha_s = fg_image[:, :, 3] / 255.0
    alpha_l = 1.0 - alpha_s
    
    output_image = np.copy(bg_image)
    for c in range(0, 3):
        output_image[y1:y2, x1:x2, c] = (alpha_s * fg_image[:, :, c] + alpha_l * bg_image[y1:y2, x1:x2, c])
        
    return output_image   

In [16]:
# whether 2 rectangle is intersected
def is_rotated_rectangle_intersected(pos1, pos2, shape1, shape2, angle1=0, angle2=0):
    # pos = (y, x), shape = (h, w), angle = 0 ~ 360
    rect1 = (pos1[::-1], shape1[::-1], angle1)
    rect2 = (pos2[::-1], shape2[::-1], angle2)
    return bool(cv2.rotatedRectangleIntersection(rect1, rect2)[0])

In [17]:
def paint_rectangle(image, rectangles):
    if len(rectangles) == 0:
        rectangles = [rectangles]
    image = image.copy()
    # line_pos = np.array(line_pos).reshape((-1, 1, 2)).astype(int) 
    for rect in rectangles:
        y_min, x_min, y_max, x_max = rect
        line_pos = np.int32([[x_min, y_min], [x_max, y_min], [x_max, y_max], [x_min, y_max]])
        cv2.polylines(image, [line_pos], True, (255, 255, 128), 3)
    return image

## Generate Train Images

In [18]:
# return image, annotation
def generate_train_image(tile_images, bg_image):
    assert len(tile_images) <= 25
    from random import uniform

    shapes  = [(uniform(64*0.9, 64*1.1), uniform(48*0.9, 64*1.1)) for img in tile_images] # (h, w)
    angles  = [uniform(0, 360) for img in tile_images]
    offsets = [] # (y, x)
    
    tile_images = [image_resize(img, shape[1], shape[0]) for img, shape in zip(tile_images, shapes)]
    tile_images = [image_rotate(img, angle) for img, angle in zip(tile_images, angles)]

    # generate non-overlapping offsets
    for i in range(len(tile_images)):
        t = 0
        while True:
            y = uniform(0, bg_image.shape[0] - tile_images[i].shape[0])
            x = uniform(0, bg_image.shape[1] - tile_images[i].shape[1])            
            confilits = [
                is_rotated_rectangle_intersected(offsets[j], (y, x), shapes[j], shapes[i], angles[j], angles[i])
                for j in range(len(offsets))
            ]
            if not np.any(confilits):
                break    
            t += 1
            if t > 20: # bg_image is too small to be non-overlapping
                i = len(tile_images)
                break
        offsets.append([y, x])
    
    result = bg_image.copy()
    for img, offset in zip(tile_images, offsets):
        result = image_overlap(result, img, offset[0], offset[1])
       
    # (y_min, x_min, y_max, x_max)
    boxes = [
        [offsets[i][0], offsets[i][1], offsets[i][0] + tile_images[i].shape[0], offsets[i][1] + tile_images[i].shape[1]]
        for i in range(len(tile_images))
    ] 
    boxes = [[int(e) for e in elem] for elem in boxes]
        
    return result, boxes

In [19]:
# voc format
def to_voc_annotation_str(filename, image_shape, labels, boxes):
# def to_voc_annotation(boxes):
    # image_shape = [h, w, c]
    # labels = [lable_name]
    # boxes = [(y_min, x_min, y_max, x_max)]
    
    objects_str = [f'''	<object>
		<name>{label}</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>{box[1]}</xmin>
			<ymin>{box[0]}</ymin>
			<xmax>{box[3]}</xmax>
			<ymax>{box[2]}</ymax>
		</bndbox>
	</object>'''
        for label, box in zip(labels, boxes)
    ]
    objects_str = '\n'.join(objects_str)
    
    annotation_str = f'''<annotation>
	<folder>ImageSets</folder>
	<filename>{filename}</filename>
	<source>
		<database>Unknown</database>
	</source>
	<size>
		<width>{image_shape[1]}</width>
		<height>{image_shape[0]}</height>
		<depth>{image_shape[2]}</depth>
	</size>
	<segmented>0</segmented>
	{objects_str}
</annotation>'''
    return annotation_str
    

In [37]:
# example:
# result, locations = generate_train_image(tile_images[1:10], bg_images[32])
# print(locations)
# result = paint_rectangle(result, locations)
# plt.imshow(result)
# plt.show()


In [28]:
# generate

train_images = []
train_images_labels = []
train_images_boxes = []

# generate 100 of 5-tiles-images
for i in tqdm(range(5)):
    sample_idxs = np.random.choice(range(len(tile_images)), size=5, replace=True)
    tile_images_sample = [tile_images[i] for i in sample_idxs]
    labels_sample = [tile_images_type[i] for i in sample_idxs]
    bg_image_sample = random.choice(bg_images)
    
    train_image, boxes = generate_train_image(tile_images_sample, bg_image_sample)
    
    train_images.append(train_image)
    train_images_labels.append(labels_sample)
    train_images_boxes.append(boxes)

    
# generate 100 of 15-tiles-images
for i in tqdm(range(2)):
    sample_idxs = np.random.choice(range(len(tile_images)), size=15, replace=True)
    tile_images_sample = [tile_images[i] for i in sample_idxs]
    labels_sample = [tile_images_type[i] for i in sample_idxs]
    bg_image_sample = random.choice(bg_images)
    
    train_image, boxes = generate_train_image(tile_images_sample, bg_image_sample)
    
    train_images.append(train_image)
    train_images_labels.append(labels_sample)
    train_images_boxes.append(boxes)
    
# generate 100 of 15-tiles-images
for i in tqdm(range(2)):
    sample_idxs = np.random.choice(range(len(tile_images)), size=20, replace=True)
    tile_images_sample = [tile_images[i] for i in sample_idxs]
    labels_sample = [tile_images_type[i] for i in sample_idxs]
    bg_image_sample = random.choice(bg_images)
    
    train_image, boxes = generate_train_image(tile_images_sample, bg_image_sample)
    
    train_images.append(train_image)
    train_images_labels.append(labels_sample)
    train_images_boxes.append(boxes)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 40.17it/s]


In [29]:
print(len(train_images))

9


In [30]:
# example: 
# i = random.choice(range(len(train_images)))
# plt.imshow((train_images[i]))
# print(train_images_labels[i])
# annotation_str = to_voc_annotation_str('00001.jpg', train_images[i].shape, train_images_labels[i], train_images_boxes[i])
# print(annotation_str)

In [31]:
# example: 
# with open('outputs/00001.xml', 'w') as file:
#     file.write(to_voc_annotation_str('0001.jpg', [480, 360, 3], ['2w', '9s'], [[1, 2, 3, 4], [2, 4, 6, 8]]))

In [32]:
# output directory

!mkdir outputs
!mkdir outputs/JPEGImages
!mkdir outputs/Annotations

mkdir: cannot create directory 'outputs': File exists
mkdir: cannot create directory 'outputs/JPEGImages': File exists
mkdir: cannot create directory 'outputs/Annotations': File exists


In [33]:
!rm -f outputs/JPEGImages/*
!rm -f outputs/Annotations/*

In [34]:
# save 

for i in tqdm(range(len(train_images))):
    img_filename = f'{i+1:06d}.jpg' # 000001.jpg
    ann_filename = f'{i+1:06d}.xml' # 000001.xml
    
    cv2.imwrite('outputs/JPEGImages/' + img_filename, train_images[i])
    
    annotation_str = to_voc_annotation_str(
        filename=img_filename, 
        image_shape=train_images[i].shape, 
        labels=train_images_labels[i], 
        boxes=train_images_boxes[i]
    )
    with open('outputs/Annotations/' + ann_filename, 'w') as file:
        file.write(annotation_str)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 115.16it/s]


Don't forget to make `ImageSet` folder and the split of dataset before use.